In [ ]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from pathlib import Path

sys.path.append("/home/acho/Sync/KiddLab/MSM/src")
from utils.stim_tools import *

sys.path.append("/home/acho/Sync/Python/sigtools")
from sigtools.representations import *
# from sigtools.sounds import *
# from sigtools.processing import *
# from sigtools.spatialization import *

In [ ]:
subject_ID = "AYC"
task_type = "SIM"
n_srcs = 3
conditions = ["co-located", "plus_minus_90", 0.5, 2, 4] # alt rate
stim_database = pd.read_csv(STIM_DIR/"stimulus_database.csv")

# Check that the specified conditions are in the stimulus database
validate_parameters(stim_database, task_type, n_srcs, conditions)

# Blocking parameters
randomize_within_block = True
n_blocks = 1                         # if randomized within block
n_trials_per_block_per_condition = 5 # if randomized within block
n_trials_per_block = None      # if NOT randomized within block
n_blocks_per_condition = None  # if NOT randomized within block
run_stim_order = set_stim_order(stim_database, task_type, n_srcs, conditions,
                                n_blocks,
                                n_trials_per_block_per_condition,
                                n_trials_per_block,
                                n_blocks_per_condition,
                                randomize_within_block)